In [4]:
import os
import json
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report,f1_score
from sklearn.model_selection import train_test_split
from openai import OpenAI


client = OpenAI(
  api_key=api_key,
)

In [ ]:
# read 
#https://platform.openai.com/docs/guides/fine-tuning/example-count-recommendations
# https://platform.openai.com/docs/guides/chat-completions/response-format
# https://github.com/openai/openai-cookbook/blob/main/examples/Fine-tuned_classification.ipynb

In [12]:
# https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

In [13]:
train = pd.read_csv('train.csv',encoding='utf-8').sample(100) # change here to use all data
test = pd.read_csv('test.csv',encoding='utf-8')

In [14]:
train

,text,label,pk
6993,E lei ha telefonato agli assistenti sociali e ...,0,7183
47,Bruxelles all'Italia: urgente ridurre il deficit.,0,49
7984,Il listino ha continuato inoltre a risentire d...,0,8231
8561,Costo dell'iniziativa: oltre un milione di dol...,0,8838
3310,"L'uomo ebbe un sorrisetto curioso, quasi che s...",0,3334
...,...,...,...
3252,Sapeva che fuori c'era il colombre ad aspettar...,1,3273
9676,Un'analoga famiglia che guadagna più del doppi...,0,10093
3238,"E nessuno a bordo scorgeva il mostro, tranne lui.",0,3255
6663,"L'Uomo, che fa il muratore ed ha diversi prece...",1,6853


In [15]:
train,val = train_test_split(train, test_size=0.20, random_state=42)

In [16]:
train['label_text'] = train['label'].apply(lambda x: 'canonical' if x==0 else 'noncanonical')
val['label_text'] = val['label'].apply(lambda x: 'canonical' if x==0 else 'noncanonical')

In [17]:
DEFAULT_SYSTEM_PROMPT = "You are an Italian language expert. You should help the user to classify the text whether its 'canonical' or 'noncanonical'."

def create_dataset(question, answer):
    return {
        "messages": [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]
    }

In [18]:
# prepare_text = """You are an Italian language expert. Analyze the following text and tell me whether it is canonical or noncanonical.
# TEXT: {text}

# Reply with either 'canonical' or 'noncanonical' as a single word.
# """.strip()


In [19]:
training_file_name = "training_data.jsonl"
validation_file_name = "validation_data.jsonl"

def prepare_data(df_data, final_file_name):
    with open(final_file_name, "w") as f:
        for _, row in df_data.iterrows():
            example_str = json.dumps(create_dataset(row["text"], row["label_text"]))
            f.write(example_str + "\n")

prepare_data(train, "training_data.jsonl")
prepare_data(val, "validation_data.jsonl")

In [20]:
training_file_id = client.files.create(
  file=open(training_file_name, "rb"),
  purpose="fine-tune"
)

validation_file_id = client.files.create(
  file=open(validation_file_name, "rb"),
  purpose="fine-tune"
)

print(f"Training File ID: {training_file_id}")
print(f"Validation File ID: {validation_file_id}")

Training File ID: FileObject(id='file-VdDnD98FgsqN1ieEGbByBmys', bytes=33340, created_at=1723757030, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Validation File ID: FileObject(id='file-9cP0xbo0eptzfBgYB62YDjQ0', bytes=7910, created_at=1723757031, filename='validation_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [21]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id.id, 
    validation_file=validation_file_id.id,
    model="gpt-3.5-turbo" #, gpt-4o-mini
#   hyperparameters={
#     "n_epochs": 4,
#     "batch_size": 5,
#     "learning_rate_multiplier": 0.3
#   }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ftjob-0Q2Q7ae8pXReC6iemfJbdUap.
Training Response: FineTuningJob(id='ftjob-0Q2Q7ae8pXReC6iemfJbdUap', created_at=1723757057, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-IHJydIZsYemK29w90Hh71BHX', result_files=[], seed=2099739449, status='validating_files', trained_tokens=None, training_file='file-VdDnD98FgsqN1ieEGbByBmys', validation_file='file-9cP0xbo0eptzfBgYB62YDjQ0', estimated_finish=None, integrations=[], user_provided_suffix=None)
Training Status: validating_files


In [22]:
import signal
import datetime

def signal_handler(sig, frame):
    status = client.fine_tuning.jobs.retrieve(job_id).status
    print(f"Stream interrupted. Job is still {status}.")
    return


print(f"Streaming events for the fine-tuning job: {job_id}")

signal.signal(signal.SIGINT, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)
try:
    for event in events:
        print(
            f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
        )
except Exception:
    print("Stream interrupted (client disconnected).")

Streaming events for the fine-tuning job: ftjob-0Q2Q7ae8pXReC6iemfJbdUap
2024-08-16 02:54:18 Validating training file: file-VdDnD98FgsqN1ieEGbByBmys and validation file: file-9cP0xbo0eptzfBgYB62YDjQ0
2024-08-16 02:54:18 Created fine-tuning job: ftjob-0Q2Q7ae8pXReC6iemfJbdUap


In [23]:
import time

status = client.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded", "failed"]:
    print(f"Job not in terminal status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = client.fine_tuning.jobs.retrieve(job_id).status
        print(f"Status: {status}")
else:
    print(f"Finetune job {job_id} finished with status: {status}")
print("Checking other finetune jobs in the subscription.")
result = client.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

Job not in terminal status: validating_files. Waiting.
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: runn

In [25]:
# Retrieve the finetuned model
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)

ft:gpt-3.5-turbo-0125:personal::9wcVGJZg


# Test using finetuned model

In [44]:
def classify_text_with_context(tuned_model,text):
    answer = client.chat.completions.create(
      model= tuned_model,
      messages= [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": text}
      ]
    )
    return answer.choices[0].message.content

In [45]:
query = test['text'][14]
classify_text_with_context(fine_tuned_model,query)

'canonical'

# Test on 20 test data

In [46]:
from tqdm import tqdm

In [47]:
pred = []
for query in tqdm(test['text'][:20]): # change here to use all test data
    result = classify_text_with_context(fine_tuned_model,query)
    pred.append(result)

100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [00:24<00:00,  1.20s/it]


In [48]:
pred = [0 if i=='canonical' else 1 for i in pred]

In [49]:
pred

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [50]:
print(classification_report(test['label'][:20],pred)) # change here to use all test data
from sklearn.metrics import f1_score
f1_score(test['label'],pred)

              precision    recall  f1-score   support

           0       0.65      1.00      0.79        13
           1       0.00      0.00      0.00         7

    accuracy                           0.65        20
   macro avg       0.33      0.50      0.39        20
weighted avg       0.42      0.65      0.51        20



C:\Users\avitr\anaconda3\envs\llm_env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\avitr\anaconda3\envs\llm_env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\avitr\anaconda3\envs\llm_env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
